<a href="https://colab.research.google.com/github/tomasplsek/AstroML/blob/main/03_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. hands-on session: **From *Data* to *Prediction***

## **Contents**

1. Preprocess the data
1. Select features & reduce dimensions
1. Cross-validate
1. Find best hyperparameters
1. Compare classifiers
1. Combine classifiers
1. Evaluate performance
1. Predict

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Our dataset**

SDSS photometry of points sources: Star vs QSO

<img src="https://cdn.mos.cms.futurecdn.net/HgaCHZDNppE6e52yeDACo6-970-80.jpg.webp" width=500 align=left>

<img src="https://earthsky.org/upl/2021/01/supermassive-black-hole-artist-e1610556964639.jpg" width=500 align=right>



### **Sloan Digital Sky Survey (SDSS)**

- photometry & spectroscopy

- fotometry 1 billion objects

- spectroscopy 6 million objects

- **stars** from our Galaxy  &  **quasars** up to $z \approx 6$

- color system `u g r i z`

<img src="https://www.astroml.org/_images/fig_sdss_filters_1.png" align=left width=500>

In [ ]:
!wget -c "https://drive.google.com/uc?id=1IoQfGFo13ZP2wTyp-xvzQvguPYhE8TWB" -O "sdss_photo.csv"

## **Data preprocessing**

In [ ]:
data = pd.read_csv("sdss_photo.csv")

In [ ]:
data

In [ ]:
data.describe().round(2)

In [ ]:
star = data.target == "star"
qso = data.target == "QSO"

sum(star), sum(qso)

In [ ]:
filters = ["u", "g", "r", "i", "z"]

plt.figure(figsize=(12,12))

x = 0
for i in range(5):
    for j in range(5):
        x += 1
        if i == j: continue
        elif i < j: continue

        f1 = filters[i]
        f2 = filters[j]

        plt.subplot(5,5,x)
        plt.plot(data.loc[qso, f2], data.loc[qso, f1], ".", label="QSOs")
        plt.plot(data.loc[star, f2], data.loc[star, f1], ".", label="stars")

        if i == 4: plt.xlabel(f2 + " [mag]")
        if j == 0: plt.ylabel(f1 + " [mag]")
        if x == 6: plt.legend()

### task 1: **create `X` and `y`**

```python
data[["u","g","r","i","z"]] -> X
data["target"] -> y
"QSO" -> 0
"star" -> 1
```

hint: you can use [LabelEncoder()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)


### task 2: **split the data (9:1), train a linear [Support Vector Classifier (SVC)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) & test its accuracy**

```python
from sklearn.svm import SVC

model = SVC(kernel="linear")
```

<img src="https://drive.google.com/uc?id=1ZyN7sykZBm0Q8xDKjMcHrezV_G91ErWH" align=left width=350>

### task 3: **rescale the data & split the data & train & test score**

note: the data were already transformed from fluxes to magnitudes `m = -log10(F/F0)`

```python
X_scaled = (X - μ) / σ
```

hint: you can use [`StandardScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

### **Pipeline**

In [ ]:
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=120, stratify=y)

model = make_pipeline(StandardScaler(),
                      SVC(kernel="linear"))

model.fit(X_train, y_train)

model.score(X_test, y_test)

In [ ]:
model

## **Feature selection & dimensionality reduction**

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
etc = ExtraTreesClassifier(random_state=42)
etc.fit(X,y)
etc.feature_importances_

In [ ]:
plt.bar(np.arange(5), etc.feature_importances_, 0.5)
plt.xticks(np.arange(5), X.columns);

#### task 4: **calculate spectral indices & test importance**


#### task 5: **test score if only *u-g* or *i-z* spectral indices are used**

hint: for single columns use `X[["u-g"]]`

#### task 6: **create dummy column & test importance**

hint:
```
X_new = X.copy()
X_new["dummy"] = np.random.randint(10, size=X.r.size)
```

## **Principal component analysis ([PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html))**

<img src="https://programmathically.com/wp-content/uploads/2021/08/pca-2-dimensions-1024x644.png" align=left width=500pt></img>

<img src="https://dimensionless.in/wp-content/uploads/2019/07/pca2.png" align=left width=750pt></img>

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_PCA = pca.fit_transform(X)
X_PCA

In [ ]:
plt.scatter(X_PCA[:,0], X_PCA[:,1], c=y);

In [ ]:
plt.scatter(X["u"], X["u-g"], c=y);

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_PCA, y, test_size=0.1, random_state=120, stratify=y)

model.fit(X_train, y_train)

model.score(X_test, y_test)

### task 7: **integrate `PCA()` into our pipeline**

Should scaling be performed before or after PCA?

## **Cross-validation ([CV](https://scikit-learn.org/stable/modules/cross_validation.html))**

<img src="https://miro.medium.com/max/1400/1*AAwIlHM8TpAVe4l2FihNUQ.png" width=800pt></img>

### task 8: **use several random states when splitting data & get average score**

### task 9: **use [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)**

## **Tuning hyperparameters**

In [ ]:
def decision_surface(X, y, classifier):
    size = 100

    X_n = PCA(n_components=2).fit_transform(X)

    range_U = np.linspace(min(X_n[:,0]), max(X_n[:,0]), size)
    range_G = np.linspace(min(X_n[:,1]), max(X_n[:,1]), size)
    U, G = np.meshgrid(range_U, range_G)

    grid = np.vstack((U.flatten(), G.flatten())).T

    classifier.fit(X_n, y)

    predicted = classifier.predict_proba(grid)[:,0]

    predicted = predicted.reshape(size,size)

    plt.contourf(U, G, predicted, cmap="coolwarm", alpha=0.5)

    plt.plot(X_n[:,0][data.target == "star"], X_n[:,1][data.target == "star"], "o")
    plt.plot(X_n[:,0][data.target == "QSO"], X_n[:,1][data.target == "QSO"], "o");

    plt.xlabel("PCA1")
    plt.ylabel("PCA2");


def classify(X, y, classifier, n_components=3):
    model = make_pipeline(StandardScaler(),
                          PCA(n_components=n_components),
                          classifier)

    score = cross_val_score(model, X, y, cv=10)
    print(np.mean(score))

In [ ]:
SVC?

In [ ]:
clf = SVC(kernel="linear", probability=True)
decision_surface(X, y, clf)
classify(X, y, clf)

In [ ]:
clf = SVC(kernel="poly", probability=True)
decision_surface(X, y, clf)
classify(X, y, clf)

In [ ]:
clf = SVC(kernel="rbf", probability=True)
decision_surface(X, y, clf)
classify(X, y, clf)

In [ ]:
clf = SVC(kernel="rbf", C=1000, probability=True)
decision_surface(X, y, clf)
classify(X, y, clf)

#### task 10: **find SVC hyperparameters with best test score**

change: n_components, kernel, C

### **Grid-search + crossvalidation**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
components = [2, 3, 4]
C = [0.01, 0.1, 1, 10, 100]

params = {"pca__n_components" : components,
          "svc__C" : C}

model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      SVC(kernel="rbf"))

gs = GridSearchCV(model, params, cv=20, n_jobs=4)
gs.fit(X, y)

In [ ]:
gs.cv_results_

In [ ]:
pars, score = gs.cv_results_["params"], gs.cv_results_["mean_test_score"]

indices = np.argsort(score)

for i in indices:
    print(pars[i], score[i].round(3))

In [ ]:
res = gs.cv_results_

plt.figure(figsize=(len(C)*1.5,len(components)*1.5))
plt.imshow(res["mean_test_score"].reshape(len(C), len(components)), origin="lower")
n = 0
for i in range(len(C)):
    for j in range(len(components)):
        plt.text(j,i,"{0:.3f}".format(res["mean_test_score"][n]), ha="center")
        n += 1

plt.ylabel("C")
plt.xlabel("n_components")
plt.yticks(np.arange(len(params["svc__C"])), params["svc__C"]);
plt.xticks(np.arange(len(params["pca__n_components"])), params["pca__n_components"]);

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
decision_surface(X, y, SVC(kernel="rbf", C=10, gamma=1, probability=True))

## **Ensemble methods**

### **Various hyper-parameters**

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
classifiers = [("Linear", SVC(kernel="linear")),
               ("Quadratic", SVC(kernel="poly", degree=2))]

model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

score = cross_val_score(model, X, y, cv=10)
np.mean(score), np.std(score)

In [ ]:
decision_surface(X, y, StackingClassifier(classifiers))

### **Random forest**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifiers = [("One", DecisionTreeClassifier(random_state=1)),
               ("Two", DecisionTreeClassifier(random_state=2)),
               ("Three", DecisionTreeClassifier(random_state=3)),
               ("Four", DecisionTreeClassifier(random_state=4))]

model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

score = cross_val_score(model, X, y, cv=10)
np.mean(score), np.std(score)

In [ ]:
model

In [ ]:
decision_surface(X, y, StackingClassifier(classifiers))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      RandomForestClassifier(n_estimators=4))

score = cross_val_score(model, X, y, cv=10)
np.mean(score), np.std(score)

In [ ]:
model

In [ ]:
decision_surface(X, y, RandomForestClassifier(n_estimators=100))

### **Stacking multiple methods**

In [ ]:
from sklearn.neural_network import MLPClassifier # multi-layer perceptron classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
classifiers = [("SVC", SVC(kernel="rbf", C=10, gamma=1)),
               ("RFC", RandomForestClassifier()),
               ("MLP", MLPClassifier(max_iter=1000)),
               ("Bayes", GaussianNB()),
               ("KNN", KNeighborsClassifier())]

for classifier in classifiers:
    print(classifier[0], classifier[1])
    classify(X, y, classifier[1])
    print()

In [ ]:
model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

score = cross_val_score(model, X, y, cv=10)
np.mean(score), np.std(score)

In [ ]:
model

In [ ]:
decision_surface(X, y, StackingClassifier(classifiers))

### **Stacking classifiers + GridSearchCV**

In [ ]:
classifiers = [("SVC", SVC(kernel="rbf")),
               ("RFC", RandomForestClassifier()),
               ("MLP", MLPClassifier(max_iter=1000)),
               ("Bayes", GaussianNB()),
               ("KNN", KNeighborsClassifier())]

components = [3, 4]
C = [0.01, 1, 100]
hidden_layer_sizes = [10, 50]

params = {"pca__n_components" : components,
          "stackingclassifier__SVC__C" : C,
          "stackingclassifier__MLP__hidden_layer_sizes" : hidden_layer_sizes}

model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

gs = GridSearchCV(model, params, cv=20, n_jobs=4)
gs.fit(X, y)

In [ ]:
gs.best_estimator_.score(X_train, y_train)

## **Performance evaluation**

### **[`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) and [`ConfusionMatrixDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html)**

In [ ]:
classifiers = [("SVC", SVC(kernel="rbf")),
               ("RFC", RandomForestClassifier()),
               ("MLP", MLPClassifier(max_iter=1000)),
               ("Bayes", GaussianNB()),
               ("KNN", KNeighborsClassifier())]

model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, display_labels=["QSO", "star"], normalize="true");

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))

## **Conclusion**

In [ ]:
# Data preparation
X = data[["u","g","r","i","z"]]
le = LabelEncoder()
y = le.fit_transform(data["target"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=420)

# Stacking classifiers
classifiers = [("SVC", SVC(kernel="rbf", C=10, gamma=1)),
               ("RFC", RandomForestClassifier()),
               ("MLP", MLPClassifier(max_iter=1000)),
               ("Bayes", GaussianNB()),
               ("KNN", KNeighborsClassifier())]

# Making a pipeline
model = make_pipeline(StandardScaler(),
                      PCA(n_components=3),
                      StackingClassifier(classifiers))

# Fitting a model for train data
model.fit(X_train, y_train)

# Testing a model on test data
score = model.score(X_test, y_test)

ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, display_labels=["QSO", "star"], normalize="true");

# Cross-validation
score = cross_val_score(model, X, y, cv=10)
print(np.mean(score), np.std(score))

## **Model inference**

In [ ]:
classifiers = [("SVC", SVC(kernel="rbf", C=10, gamma=1)),
               ("RFC", RandomForestClassifier()),
               ("MLP", MLPClassifier(max_iter=1000)),
               ("Bayes", GaussianNB()),
               ("KNN", KNeighborsClassifier())]

model = make_pipeline(StandardScaler(),
                      PCA(n_components=2),
                      StackingClassifier(classifiers))

model.fit(X, y)

### task 11: **pick an object from SDSS and classify it**

http://skyserver.sdss.org/dr7/en/tools/search/radial.asp

In [ ]:
u = 
g = 
r = 
i = 
z = 

X_real = pd.DataFrame(np.array([[u,g,r,i,z]]), columns=["u","g","r","i","z"])

X_real